In [50]:
import json
l = ['1.json',
     '2.json',
     '3.json']
dicts = []
for el in l:
    with open(el, 'r') as f:
        
        dicts.append(json.load(f))
        print(len(dicts[-1]))
len(dicts)

16073
20853
12935


3

In [44]:
ans = {}
for el in dicts:
    for k in el.keys():
        ans[k] = el[k]
# если длина меньше 49к, сабмитить нет смысла. НУжно делать предикт по всем    
len(ans)

49394

In [37]:
import json
with open('my_best.json', 'w') as f:
    json.dump(ans, f)

## Объединение двух json'ov

In [33]:
import numpy as np
import json
d1 = {}
d2 = {}
with open('./share/2_5312170359494017878.json', 'r') as f:
    d2 = json.load(f)
    
with open('./share/nn_light_bm.json', 'r') as f:
    d1 = json.load(f)

In [34]:
len(d2)

50000

In [35]:
av = []
d = {}
counter = 0

for key in d1.keys():
    if key in d2.keys():
        ar1 = np.array(d1[key])
       # print(ar1)
        ar2 = np.array(d2[key])
        inter = np.intersect1d(ar1,ar2)
        av.append(len(inter))
        to_add = []
        for el in ar2:
            if el  in inter:
                to_add.append(el)
        for el in ar2:
            if el  not in to_add:
                to_add.append(el)
        
        
        to_add = np.array(to_add)
        
        d[key] = list(map(int,list(to_add)))

    else:
        d[key] = d1[key]

  

#with open('1.json', 'w') as f:
#    json.dump(d, f)
    
print('average intersaction equals {},{}'.format(np.array(av).mean(),np.array(av).var()))

average intersaction equals 9.309129171259645,7.664415104970264


In [9]:
len(d)

49895

## Женин метод

In [12]:
d1 = {}
d2 = {}
d3 = {}
#with open('./share/predskazania_setki_all.json', 'r') as f:
#    d2 = json.load(f)
    
with open('./share/light_fm_1604.json', 'r') as f:
    d2 = json.load(f)
    
with open('./share/16_set_to_submit.json', 'r') as f:
    d1 = json.load(f)

In [13]:
new = set(d1).union(set(d2))#.union(set(d3))

In [8]:
from collections import OrderedDict,defauldict
result = {}
for user in new:
    r1 = False
    r2 = False
    r3 = False
    try:
        ar1 = d1[user]
        r1 = True
    except:
        pass
    try:
        ar2 = d2[user]
        r2 = True
    except:
        pass
#    try:
#        ar3 = d3[user]
#        r3 = True
#    except:
#        pass
    all_films = list(set(ar1).union(set(ar2)))#.union(set(ar3)))
  #  print(all_films)
    f_dict ={}
    for el in all_films:
        rank = 0
        to_drop= 0
        if el in ar1:
            rank += ar1.index(el)
        else:
            rank +=21
        if el in ar2:
            rank += ar2.index(el)
        else:
            rank +=21
    #    if el in ar3:
    #        rank += ar3.index(el)
    #    else:
    #        rank +=21
        f_dict[el] = rank
        
    t = sorted(f_dict.items(), key=lambda x: x[1])
    to_add =[]
    for el in t:
        to_add.append(el[0])
    result[user] = to_add[:20]
   # print(result[user])
        


    

ImportError: cannot import name 'defauldict'

In [7]:
len(d2)

50000

In [9]:
from collections import defaultdict

In [15]:
with open('blend.json', 'w') as f:
    json.dump(result, f)

result

In [84]:
len(result["0"])

20

In [105]:
set([1,2,3,4]).union([2,3,4,55])

{1, 2, 3, 4, 55}

In [77]:
with open('./share/bm25_last_submit.json', 'r') as f:
    d2 = json.load(f)
    
with open('./share/blend.json', 'r') as f:
    d1 = json.load(f)
    
with open('./share/answer_50.json', 'r') as f:
    d4 = json.load(f)
    
with open('./share/rf.json', 'r') as f:
    d5 = json.load(f)
    
    
    
with open('./share/2_5312170359494017878.json', 'r') as f:
    d3 = json.load(f)

In [78]:
sol_merged = dict()
lens = []
for user, els1 in d1.items():
    merget_score = defaultdict(int)

    els2 = d2[user]
    els3 = d3[user]
    els4 = d4[user]
    els5 = d5[user]
    union_el = set(els1).union(els2).union(els3).union(els4).union(els5)
    for i, el2 in enumerate(els2):
        merget_score[el2] += i- 1
    for el in union_el.difference(els2):
        merget_score[el] += 19
        
    for i, el3 in enumerate(els3):
        merget_score[el3] += i
    for el in union_el.difference(els3):
        merget_score[el] += 20
        
    for i, el4 in enumerate(els4):
        merget_score[el4] += i+1
    for el in union_el.difference(els4):
        merget_score[el] += 21
        
    for i, el5 in enumerate(els5):
        merget_score[el5] += i+2
    for el in union_el.difference(els5):
        merget_score[el] += 22
        
        
    current_top = list(zip(*sorted(merget_score.items(),key=lambda x:x[1])))[0]
    lens.append(len(current_top))
    sol_merged[user] = current_top[:20]

In [79]:
with open('./share/final.json', 'w') as f:
    json.dump(sol_merged, f)

In [80]:
validation(sol_merged)

0.5814547965520875

In [44]:
def validation(dict_):
    d1 = {}
    with open('./share/2_5312170359494017878.json', 'r') as f:
        d1 = json.load(f)
    av = []


    for key in d1.keys():
        if key in dict_.keys():
            ar1 = np.array(d1[str(key)])
         # print(ar1)
            ar2 = np.array(dict_[str(key)])
            inter = np.intersect1d(ar1,ar2)
        #  av.append(len(inter))
            av.append(mapk([ar1],[ar2]))
            to_add = []
            for el in ar1:
                if el  in inter:
                    to_add.append(el)
            for el in ar1:
                if el  not in to_add:
                    to_add.append(el)


            to_add = np.array(to_add)
    return np.mean(av)



def mapk(actual, predicted, k=20):
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])
  
def apk(actual, predicted, k=10):

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

   # if not actual:
   #     return 0.0

    return score / min(len(actual), k)

In [51]:
validation(d3)

0.11192012412112942